# Deploy LLaVA on Amazon SageMaker

Amazon SageMaker is a popular platform for running AI models, and models on huggingface deploy [Hugging Face Transformers](https://github.com/huggingface/transformers) using [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html) and the [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/).

![llava](https://i.imgur.com/YNVG140.png)

Install sagemaker sdk:

In [ ]:
!pip install sagemaker --upgrade
!pip install -r code/requirements.txt

Bundle llava model weights and code into a `model.tar.gz`:

In [ ]:
# Create SageMaker model.tar.gz artifact
!tar -cf model.tar.gz --use-compress-program=pigz *

After we created the `model.tar.gz` archive we can upload it to Amazon S3. We will use the `sagemaker` SDK to upload the model to our sagemaker session bucket.

Initialize sagemaker session first:

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    # setup your own rolename in sagemaker
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231008T201275')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

Upload the `model.tar.gz` to our sagemaker session bucket:

In [ ]:
from sagemaker.s3 import S3Uploader

# upload model.tar.gz to s3
s3_model_uri = S3Uploader.upload(local_path="./model.tar.gz", desired_s3_uri=f"s3://{sess.default_bucket()}/llava-v1.5-13b")

print(f"model uploaded to: {s3_model_uri}")

We will use `HuggingfaceModel` to create our real-time inference endpoint:

In [ ]:

from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_model_uri,      # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.28.1",  # transformers version used
   pytorch_version="2.0.0",       # pytorch version used
   py_version='py310',            # python version used
   model_server_workers=1
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    # container_startup_health_check_timeout=600, # increase timeout for large models
    # model_data_download_timeout=600, # increase timeout for large models
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/tom/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/tom/Library/Application Support/sagemaker/config.yaml
---------------!

The `.deploy()` returns an `HuggingFacePredictor` object which can be used to request inference using the `.predict()` method. Our endpoint expects a `json` with at least `image` and `question` key.

In [ ]:
data = {
    "image" : 'https://raw.githubusercontent.com/haotian-liu/LLaVA/main/images/llava_logo.png',
    "question" : "Describe the image and color details.",
    # "max_new_tokens" : 1024,
    # "temperature" : 0.2,
    # "stop_str" : "###"
}

# request
output = predictor.predict(data)
print(output)

The image is a black and white photograph of a man standing in front of a building. The man is wearing a suit and tie, and he appears to be looking at the camera. The building in the background is large and has many windows. The overall atmosphere of the image is formal and professional.


To run inference with `llava` special token:

In [ ]:
from llava.conversation import conv_templates, SeparatorStyle
from llava.constants import (
DEFAULT_IMAGE_TOKEN,
DEFAULT_IM_START_TOKEN,
DEFAULT_IM_END_TOKEN,
)
def get_prompt(raw_prompt):
    conv_mode = "llava_v1"
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles
    inp = f"{roles[0]}: {raw_prompt}"
    inp = (
        DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + "\n" + inp
    )
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    return prompt, stop_str

raw_prompt = "Describe the image and color details."
prompt, stop_str = get_prompt(raw_prompt)
image_path = "https://raw.githubusercontent.com/haotian-liu/LLaVA/main/images/llava_logo.png"
data = {"image" : image_path, "question" : prompt, "stop_str" : stop_str}
output = predictor.predict(data)
print(output)

The image features a red toy animal, possibly a horse or a donkey, with a pair of glasses on its face. The toy is made of plastic and has a fire-like appearance, giving it a unique and eye-catching look. The red color of the toy and the glasses on its face create a striking contrast against the background, making it the main focus of the image.


The inference ` predictor` can also be initilized like with your deployed `endpoint_name` :

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    # setup your own rolename in sagemaker
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231008T201275')['Role']['Arn']

from sagemaker.huggingface.model import HuggingFacePredictor
# initial the endpoint predictor
predictor2 = HuggingFacePredictor(
    endpoint_name="huggingface-pytorch-inference-2023-10-19-05-57-37-847",
    sagemaker_session=sess
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/tom/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/tom/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name arn:aws:iam::297308036828:root to get Role path.


In [ ]:
raw_prompt = "Describe the image and color details."
prompt, stop_str = get_prompt(raw_prompt)
image_path = "https://raw.githubusercontent.com/haotian-liu/LLaVA/main/images/llava_logo.png"
data = {"image" : image_path, "question" : prompt, "stop_str" : stop_str}
output = predictor2.predict(data)
print(output)

The image features a small toy animal, resembling a horse or a donkey, with a red and orange color scheme. The toy has a pair of glasses on its face, giving it a unique and quirky appearance. The toy is standing on a gray surface, which provides a contrasting background for the vibrant colors of the toy. The combination of red, orange, and gray creates a visually striking scene.


To clean up, we can delete the model and endpoint by `delete_endpoint()`or using sagemaker console:

In [ ]:
# delete sagemaker endpoint
predictor.delete_endpoint()